In [12]:
###########################                         PATRONS                    ######################################## 
#The following bits of code are used to send emails to patrons informing them that their specific 3D print is completed and ready for pickup 

#Full code is found in email_patrons.py but I just used below to break it up into meaningful chunks to troubleshoot and readability  

In [ ]:
import pandas as pd 

#Importing in master 3D printing data set from onedrive to get updated form data 
excel_file_master = r"~\Desktop\Example data.xlsx"

df = pd.read_excel(excel_file_master)

#filtering data by the people who's print is completed, they have not been contacted yet, they do not have an invalid email address, and there print has not been picked up yet
df_filter = df[(df["print_completed"] == "X") & (df["patron_contacted"].isna()) & (df["invalid_email"].isna()) & (df["picked_up"].isna())] #can do is.na or set patron_contacted = "X"

df_filter  

In [ ]:
#Following code applies regex to a list of email addresses we gathered above then sorts by validness 
import re

#Gathering the list of emails from the data set who's prints have been completed (but not yet contacted) so that they can now be emailed 
emails = df_filter["email"].tolist()

#Creates 2 empty lists where I will store valid and invalid email addresses based on output of regex implementation  
valid_emails = []
not_valid_emails = []

for email in emails: # iterate through each email address in the list gathered from the data set 
    if re.search("^(\w+[\.\-]?)+@+\w+\.(com|org|edu|net)$", email, re.IGNORECASE):
        
        # Use the re.search function to search for a match with the regular expression pattern in the email string
        # The regular expression pattern is surrounded by quotes and passed as the first argument to the re.search function
        # The email address we're matching against is passed as the second argument, in this case what we're iterating over 
        # The re.IGNORECASE flag is used to make the search case-insensitive

        valid_emails.append(email) #If the search function returns true (aka email passes) then the email gets added to the valid emails list 
        print(f"{email} is a valid email")

    else:
        not_valid_emails.append(email) # If email does not match regex, then that email is added to the not_valid_emails list
        print(f"{email} is NOT a valid email!!")

#What the regualr expression above does:
    #So, the regular expression above is looking for a string that starts with one or more word characters (alphanumeric characters (letters and digits) and Underscores (_)) 
    # or a word character followed by an optional period or hyphen, followed by an @ symbol,followed by one or more word characters, followed by a period, and then either com, org, or edu. 
    # The whole string must match the pattern, and no additional characters can be present at the end.

#The \n denotes that I want a space before and after this print statement is executed to make reading it in the command line easier 
print("\nThe following email addresses are valid and an email will be sent to them saying their specific 3D print is completed and ready for pickup:") 
print(valid_emails)

print("\nThe following email addresses were flagged for being invalid and no email will be sent to them:")
print(not_valid_emails)

In [ ]:
#Taking the list of invalid emails and adding that information into the invalid_email column for our updated data frame
# df_filter.loc is a locator method to find the row(s) where email column equals the current invalid email
for invalid_email in not_valid_emails:
    df_filter.loc[df_filter["email"] == invalid_email, "invalid_email"] = "X" #Essentially, when value in email variable is in the not_valid_emails list, we add an "X" in the invalid_email variable to say 
                                                                                #that observation contains an invalid email. 

print("\nThe invalid_email varaible has been updated with the above invalid emails\n")

In [ ]:
df_filter

In [ ]:
#filter out invalid emails from the data frame to send out emails that properly correspond to their print  
df_filter_valid = df_filter[(df_filter["invalid_email"] != "X")]
df_filter_valid #Means the data is filtered (columns and observations) + only the valid emails are listed 

In [ ]:
#Prompt in the middle of the code making sure that we want to contine sending the emails out to patrons 
import sys

user_input = input("Seeing the valid and invalid email addresses, would you like to continue with sending emails to the valid email addresses? (y/n) ")

if user_input.lower() == "y":
    print("Continuing with the program.\n")
else:
    print("Exiting the program.")
    sys.exit()

In [ ]:
import datetime

today_str = datetime.date.today().strftime("%m/%d/%y") #Gets today's date in the format of "02/26/23"

#Converting the date string we got above into an actual date  
today_date = datetime.datetime.strptime(today_str, "%m/%d/%y").date()

#We use the .fillna() method to replace all empty values in patron_contacted with today's date regardless of whether email is valid or not  
df_filter.loc[df_filter['contacted_date'].isna(), 'contacted_date'] = today_date

df_filter

In [ ]:
#Code to send emails to patrons informing them that their specific 3D print is completed and ready for pickup 

from email.message import EmailMessage # Library that formats code and strings as an email 
import ssl # Adds security to our email 
import smtplib # Library that actually sends the email 
import time  #used to get time.sleep function pause the program as to not send emails too fast and get blocked by gmail

email_sender = ""
email_pass = "XXXXXXXXXXX"
email_receivers = valid_emails.copy()

# Extract the print variables into separate lists
prints = df_filter_valid["print"].tolist()

#Defines subject header for emails 
subject = "3D Print is Ready for Pickup!"

# Adding a layer of security to our email 
context = ssl.create_default_context()

for i in range(len(email_receivers)):
    #What we want our message in the email to say 
    body = "Hello," + "\n\n" + f"Your {prints[i]} print has been completed and is ready for pickup!" + \
    "\n\n" + f" Please pick up your {prints[i]} print within 2 weeks at the lower level desk of the library during open hours." + \
    "\n\n" + "From the library 3D Printing Team"
    
    # Matching up the parameters above for our email to the format python expects them 
    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receivers[i]
    em['Subject'] = subject
    em.set_content(body)
    
    #Opening the connection to the gmail sever, logging in and then sending the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_pass)
        smtp.sendmail(email_sender, email_receivers[i], em.as_string())
        
        #Adding a nice print step confimring an email has been sent to all of the email addresses 
        print("Email successfully sent to " + email_receivers[i])

        # Adds an "X" in the patron_contacted column for the email addresses we just sent messages to
        df_filter.loc[df_filter["email"] == email_receivers[i], "patron_contacted"] = "X"
        
    #Sleeping the program for 2 seconds to avoiding sending too many emails back to back and getting timed out by gmail
    time.sleep(2)


#Print statement to let me know that the program is done     
print("All emails sent successfully!\n")

In [ ]:
import os 

#Recreating the folder to save excel files if it doesn't already exist, os.path.expanduser is used because os.makedirs doesn't like the ~
home_dir = os.path.expanduser("~")
#Specifies the directory we're going to search for files and create the new excel file noting patrons contacted 
directory_save = os.path.join(home_dir, "Desktop", "3D Printing output files")
os.makedirs(directory_save, exist_ok=True)

string = "Patron_Contacted_Updated"  #This string will be in the excel file we're searching for to either overwrite it or not  

today = datetime.datetime.now().strftime("%b %d") #Gets today's today in the format of "Feb 18"

file_name = f"Patron_Contacted_Updated {today}.xlsx" #Creating a new excel file with the updated date in the name 

file_path = os.path.join(directory_save, file_name) #Combining the directory and file name to create the file path to more easily write this file location later on 


for filename in os.listdir(directory_save): #loops through all the files in the directory to search for the excel file with "Patron_Contacted_Updated" (if it exists) then update it!
    if string in filename:
        
        file_pathy = os.path.join(directory_save, filename)
        answer = input(f"Old version of this file named {filename} exists, would you like to delete it? (y/n) ")

        if answer == 'y': #Code to remove the old excel file directed 
            os.remove(file_pathy)
            print(f"\'{filename}\' has been deleted\n")

        else:
            print(f"The existing file \'{filename}\' will not be overwritten.")
            sys.exit()
        

#Code to create a new excel doc marking the patrons who have been contacted via email
master_df = pd.read_excel(excel_file_master) #loads in master excel doc from onedrive

# changing the contacted_date variable from a datetime to just a date
master_df['contacted_date'] = master_df['contacted_date'].dt.date

master_df.update(df_filter) #updates master_df doc with the updated patron_contacted varaible from df_filter (AKA tallying patrons who have been contacted)

master_df.to_excel(file_path , index=False) #Saves the updated master_df to onedrive to be used to update the master excel doc (that the form feeds into)

print(f"New file: \'{file_name}\' has successfully been created!\n")

#After the emails are sent we need to update the patron_contacted column so this will create another excel I'll use to update the master doc 
print(f"Now, use \'{file_name}\' to update the varaibles 'patron_contacted', 'contacted_date', 'invalid_email' in the Master 3D Printing doc!!!")

time.sleep(6)

In [7]:
#########################                   3D print sheet for Staff                ######################################

#Code to send email to the 3D print team each with excel file that will have sorted the 3D prints to be printed for that week. Staff then prints this off at work

In [ ]:
import pandas as pd 
import os
import datetime

excel_file_master = r"~\Desktop\Example data.xlsx"

#Reading in the updated data from the microsoft form using an excel doc stored on onedrive 
df = pd.read_excel(excel_file_master) 

#making sure python understands this excel file is a data frame 
df = pd.DataFrame(df)

#Filtering the data frame by patrons who's print has not ealready been completed and who's print has has not already gotten on the physical sheet  
df = df[(df["print_completed"] != "X") & (df["sheet_print"] != "X") ]

# format the 'number' column as a floating point variable with 0 decmial places (NOT WORKING)
#df['card_number'] = df['card_number'].apply(lambda x: format(x, '.0f'))

df

In [ ]:
#Selecting specific varaibles to be displayed in the excel file so that it's easier to work with when printed 

#Select by selecting
#df_filtered_selected = df_filtered[["ID" ,"completion_time", "name", "email", "phone_number", "print", "color"]]

#Select by dropping 
#Drop specific variables from our data frame
df_selected = df.drop(columns= ["patron_contacted", "contacted_date", "invalid_email", "print_expired", "sheet_print", "recontacted?"])

df_selected

In [ ]:
#Sorting the base data frame primarily by color and then by date for easy read ability 

df = pd.DataFrame(df)

df_selected_sorted = df_selected.sort_values(by=['color', 'completion_time'],  ascending= [False, True])

df_selected_sorted

In [ ]:
#simplifying the datetime variable completion_time to make it more readable to staff 

df_selected_sorted['completion_time'] = pd.to_datetime(df_selected_sorted['completion_time'])
df_selected_sorted['completion_time'] = df_selected_sorted['completion_time'].dt.strftime("%b %d, %Y") #puts date in the format: "Feb 18, 2023"

df_selected_sorted

In [ ]:
#Initiating a prompt in the program to see if the user wants to continue with creating an excel file 
import sys

print("\nBelow are the first 10 observations of the '3D Prints to be Completed' DataFrame:\n")
print(df_selected_sorted.head(10))

while True:
    answer = input("\nSeeing the above DataFrame, would you like to make it an excel file and save it locally? (y/n) ")
    if answer == "y":
        break
    elif answer == "n":
        print("The program will now cease running.")
        time.sleep(3)
        sys.exit()
    else:
        print("Invalid input. Please enter 'y' to continue or 'n' to exit.")

In [ ]:
#The following code saves the df_selected_sorted data frame I've edited and saves it locally as an excel doc 

today = datetime.datetime.now().strftime("%b %d") #gets the current datatime at the moment the code is run, formats it with strftime as 'Feb 20'

#Save excel file with name based on day program is run
file_name = f"3D print jobs for week of {today}.xlsx"

#Recreating the folder to save excel files if it doesn't already exist, os.path.expanduser is used because os.makedirs doesn't like the ~
home_dir = os.path.expanduser("~")
directory_save = os.path.join(home_dir, "Desktop", "3D Printing output files")
os.makedirs(directory_save, exist_ok=True)

#Creating the complete path (location) where I want to save the new excel file
file_path = directory_save + "\\" + file_name

#Code to see if the file_path already exists then if it does asks user if they want to overwrite it 
if os.path.exists(file_path):
    answer = input(f"\n{file_name} already exists, would you like to overwrite it with the new file? (y/n) ")

    if answer == 'y': 
        os.remove(file_path)
        print(f"\n\"{file_name}\" has successfully been saved\n")

    else:
        print(f"Old {file_name} file will not be overwritten")
        print("This program will now cease running...")
        time.sleep(3)
        sys.exit()

else:
    print(f"\n\"{file_name}\" has successfully been saved")

#Code to saved the excel file to the file_path specified above 
df_selected_sorted.to_excel(file_path, index=False) 

In [ ]:
#Making a separate excel sheet marking which prints have been added to the physical printing sheet 

directory = r"~\Desktop\3D Printing output files" #Specifies the directory we're going to search for files and create the new excel file noting patrons contacted  
today = datetime.datetime.now().strftime("%b %d") #Gets today's today in the format of "Feb 18"
file_name2 = f"Updated sheet_print {today}.xlsx" #Creating a new excel file with the updated date in the name 
file_path2 = os.path.join(directory, file_name2) #Combining the directory and file name to create the file path to more easily write this file location later on 

#we need to read in the orginal file again to a dataframe to get the structure of it making copy and pasting the inputs back in way easier 
#Code to create a new excel doc marking the patrons who have been contacted via email
master_df = pd.read_excel(excel_file_master) #loads in master excel doc from onedrive

#Getting the list of varaibles that are on our 3D print list we want to keep track of 
identification = df_selected_sorted["ID"].tolist()

#Takes all the IDs of the patrons who are being put on the 3D print list for the week and puts an X in the sheet_print column noting that they've been printed
for id in identification:
   master_df.loc[master_df["ID"] == id, "sheet_print"] = "X" 

master_df.to_excel(file_path2 , index=False) #Saves the updated master_df to onedrive to be used to update the master excel doc (that the form feeds into)

print(f"\nAlso, new file: \'{file_name2}\' has successfully been created!\n")

print(f"Now, use \'{file_name2}\' document to update the varaible 'sheet_print' in the Master 3D Printing doc!!!\n")

time.sleep(4) 

In [ ]:
#Another prompt now that the excel file is saved locally seeing if we want to email it out to 3D print staff  

user_input = input(f"Now that {file_name} is saved locally, would you like to email it out to 3D print staff? (y/n) ")

if user_input.lower() == "y":
    print("Prepping email to print to staff!\n")
else:
    print("With the file just saved locally, make sure to print it out!")
    time.sleep(4)
    sys.exit()

In [ ]:
#Code to generate a random quote from the API link to add at the end of the weekly email

import requests
import json

def get_random_quote():
  url = "http://api.quotable.io/random"
  response = requests.get(url)
  if response.status_code == 200:
    quote = response.json()
    return quote["content"] + " - " + quote["author"]
  else:
    return "Error: Could not retrieve quote."

quote_of_the_week = 'Quote of the week: "' + get_random_quote() + '"'

quote_of_the_week 

In [ ]:
#Code to send an email to multiple email addresses at once with an excel attachment   

from email.message import EmailMessage #Library that formats code and strings as an email 
import ssl #adds security to our email 
import smtplib #library that actually sends the email 
import time  #used to get time.sleep function pause the program as to not send emails too fast and get blocked by gmail


#List of emails for you'd like to send expired print list to 
email_list = ["jack@email.com", "Max@email.com", "bobby@email.com"]

email_sender = "myemail@email.com"
email_pass = "XXXXXXXXXXXXX"

subject = f"3D Print List for Week of {today}"

body = "Hello 3D Print Team," + "\n\n" + "Please print the attatched excel file containing the 3D prints to be completed for this week." + "\n\n" + "Printing instructions: " + \
    "Expand each of the columns so that each column name and their associated observations are readable. To make this easier highlight all of the column names and left align them." + \
    " Next, under page layout, set the top, bottom, left, and right margins close to 0. Set the page orientation to 'landscape'. Then finally, click the print gridlines option. " + "\n\n" + \
    f"{quote_of_the_week}" + "\n\n" 

#defining the path of the excel attachment to be sent 
new_week_excel = file_path

# Attaching the .xlsx file to the email
with open(new_week_excel, "rb") as f:
    file_data = f.read() #File needs to be read in as binary code 

#adding a layer of security to our email 
context = ssl.create_default_context()

#matching up the parameters above for our email to the format python expects them 
em = EmailMessage()
em['From'] = email_sender
em['To'] = email_list
em['Subject'] = subject
em.set_content(body)
em.add_attachment(file_data, maintype="application", subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet", filename= file_name)

# Sending an email to all 3D print staff all at once
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(email_sender, email_pass)
    smtp.sendmail(email_sender, email_list, em.as_string())

print("\nEmail successfully sent to 3D printing team!")
time.sleep(3)

In [ ]:
#########################                   Expired 3D Print Sheet for staff                ######################################

##Code to send an expired  pick up list excel sheet to 3D print when neccessary 
#I either could just run an email script to reemail patrons or this could turn into code to generate an excel sheet when prints need to be discard 

In [ ]:
import pandas as pd 
import os as os
import datetime as dt
import time

excel_file_master = r"~\Desktop\Example data.xlsx"

df = pd.read_excel(excel_file_master)

#making sure python understands this excel file is a data frame 
df = pd.DataFrame(df)
df 

In [ ]:
#Code trying to simplify the datetime variable to make it more readable to staff 

df['completion_time'] = df['completion_time'].dt.strftime("%b %d, %Y")

df

In [ ]:
#Filtering the data frame by prints that have been completed and patrons contacted but the print has not been picked up yet and is not already marked as expired 

df = df[(df["print_completed"] == "X") & (df["patron_contacted"] == "X") & (df["picked_up"].isna()) & (df["print_expired"].isna())]
df

In [ ]:
#Selecting specific varaibles to be displayed in the excel file so that it's for staff to see when looking at expired prints page  

#Select by selecting
#df = df[["ID" ,"completion_time", "name", "email", "phone_number", "print", "color"]]

#Select by dropping 
#Drop specific variables from our data frame
df = df.drop(columns= ["print_started", "invalid_email", "sheet_print", "failed"])

df

In [ ]:
#Next we filter the contacted date so that three weeks has passed since the print has been finished but not picked up yet 

# convert the 'contacted_date' column to a datetime object
#df['contacted_date'] = pd.to_datetime(df['contacted_date'])

# calculate the date three weeks ago
three_weeks_ago = dt.datetime.now() - dt.timedelta(days=21)

# This will give you all the rows where 'contacted_date' is more than three weeks old.
df_filter = df[df['contacted_date'] < three_weeks_ago]

df_filter

In [ ]:
#A copy warning appears here but it doesn't matter so just deal with it 
#We then used the .fillna() method to replace all empty values with "X". The value parameter specifies the value to use for filling in the empty values, 
#print("\nIgnore the following warning: 'Setting With Copy Warning', it's fine \n")
df_filter.loc[df_filter['patron_contacted'] == 'X', 'print_expired'] = 'X'   


#'print_expired'].fillna(value='X', inplace=True) #the inplace parameter specifies that the changes should be made directly to the DataFrame. Python doesn't like the fillna method when used on 
# slices of a dataframe
df_filter

In [ ]:
#Showing the user the first 10 observations of the Expired 3D Prints List for verificiation 
import sys

print("\nBelow are the first 10 observations of the 'Expired 3D Prints List' DataFrame:\n")
print(df_filter.head(10))

while True:
    answer = input("\nSeeing the above DataFrame, would you like to continue with the program to save the above DataFrame locally as an excel file? (y/n) ")
    if answer == "y":
        print("Continuing with the program!") 
        break
    elif answer == "n":
        print("The program will now cease running.")
        sys.exit()
    else:
        print("Invalid input. Please enter 'y' to continue or 'n' to exit.")

# Continue running the program here

In [ ]:
#The following code takes the df_filter dataframe that contains all the expired prints for people who have been completed longer than two weeks ago and saves it locally as an excel file

#Gets today's datetime value
today = dt.datetime.now().strftime("%b %d") #The format codes %b and %d result in "Feb 20"

string = "Updated_expired_Var"  #String we are searching all files in the desktop for  

#File name changes based on the day that the program is run 
file_name = f"Updated_expired_Var {today}.xlsx"


#Recreating the folder to save excel files if it doesn't already exist, os.path.expanduser is used because os.makedirs doesn't like the ~
home_dir = os.path.expanduser("~")
#stating the directory of where to save our new expired prints excel 
directory_save = os.path.join(home_dir, "Desktop", "3D Printing output files")
os.makedirs(directory_save, exist_ok=True)

#Specific path of where I want to save the new excel doc being created 
file_path_upadte = directory_save + "\\" + file_name

#Creating the second excel file that staff wil use to easily contact the affected people  
file_print = f"Expired 3D prints week of {today}.xlsx"

file_path_print = directory_save + "\\" + file_print

#loops through all the files in the directory to search for an older version of this document (file_name) (if it exists) then update it! 
for filename in os.listdir(directory_save): 
    if string in filename:
        file_pathy = os.path.join(directory_save, filename)
        answer = input(f"\nOld file exists, \'{filename}\', would you like to delete the this old one and save the updated versions? (y/n) ")

        if answer == 'y':
            os.remove(file_pathy)
            print(f"\nOld file: \'{filename}\' has been deleted\n")

        else:
            print(f"\nThe existing files will not be overwritten.")
            print("This program will now cease running.")
            sys.exit()

#Creates the new excel file if one is not already there 

#Code to create a new excel doc marking the patrons who have been contacted via email
master_df = pd.read_excel(excel_file_master) #loads in master excel doc from onedrive

master_df.update(df_filter) #updates master_df doc with the updated patron_contacted varaible from df_filter (AKA tallying patrons who need to be recontacted)

master_df.to_excel(file_path_upadte , index=False) #Saves the updated master_df to onedrive to be used to update the master excel doc (that the form feeds into)

#After the emails are sent we need to update the patron_contacted column so this will create another excel I'll use to update the master doc 
print(f"Use \'{file_name}\' excel sheet to update the 'print_expired' varaible in the Master 3D Printing doc!!!\n")

df_filter.to_excel(file_path_print , index=False)

print(f"Use \'{file_print}\' excel sheet to identify and remove the 3D prints that have been sitting for 3 weeks after the patron was initally contacted")

In [ ]:
#Another prompt now that the excel file is saved locally seeing if we want to email it out to 3D print staff  

user_input = input(f"\nNow that \'{file_name}\' and \'{file_print}\' are saved locally, would you like to email them to 3D print staff? (y/n) ")

if user_input.lower() == "y":
    print("Prepping email to print to staff!\n")
    time.sleep(1)

else:
    print("With the file just saved locally, make sure to print it out!")
    time.sleep(4)
    sys.exit()

In [ ]:
#Code to send an email with multiple attachments to multiple email addresses at once  

from email.message import EmailMessage #Library that formats code and strings as an email 
import ssl #adds security to our email 
import smtplib #library that actually sends the email 
import time  #used to get time.sleep function pause the program as to not send emails too fast and get blocked by gmail

 
#List of emails for you'd like to send expired print list to 
email_list = ["jack@email.com", "Max@email.com", "bobby@email.com"]

#Email credentials 
email_sender = "myemail@email.com"
email_pass = "XXXXXXXXXXXXXX"

subject = f"Expired Prints List for Week of {today}"

body = "Hello 3D Print Team," + "\n\n" + "The attached excel file contains the list of expired prints we can donate or discard. " + \
    "\n\n" + "Patrons who were contacted more than 3 weeks ago about their print and have not yet picked it up are on this list.\n" 

# list of file paths to attach to the email 
file_paths = [file_path_upadte, file_path_print]

#matching up the parameters above for our email to the format python expects them 
em = EmailMessage()
em['From'] = email_sender
em['To'] = email_list
em['Subject'] = subject
em.set_content(body)

# Attaching the .xlsx file to the email
for file_path in file_paths:
    # get the file name from the file path
    file_name = os.path.basename(file_path)
    with open(file_path, "rb") as f:
        file_data = f.read() #File needs to be read in as binary code 
        em.add_attachment(file_data, maintype="application", subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet", filename= file_name)

#adding a layer of security to our email 
context = ssl.create_default_context()

# Sending an email to all 3D print staff all at once
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(email_sender, email_pass)
    smtp.sendmail(email_sender, email_list, em.as_string())

print(f"\n\'{file_name}\' and {file_print} has been successfully emailed to the 3D printing team!")